In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import findspark
findspark.init('/home/rajdeep/spark-3.5.0-bin-hadoop3/')

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
#initalizing the spark object
spark = SparkSession.builder.appName('wordCount').getOrCreate()

In [6]:
#create a socket to trasmit data
# in unix shell - nc -lk [port]
# for e.g., nc -lk 9999

In [7]:
#reading the data from console for stream processing
lines = spark.readStream.format("socket").option("host","localhost").option("port","9999").load()

23/12/27 16:20:53 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [8]:
#printing the schema
lines.printSchema()

root
 |-- value: string (nullable = true)



In [9]:
#splitting the input lines into words
word = lines.select(explode(split(col('value'),'\s+')).alias("words"))

In [10]:
#counting the occurence of each words
count = word.groupBy("words").count()

In [11]:
#initalizing stream output details
streamingquery = count.writeStream.format("console").outputMode("complete").trigger(processingTime="1 second").start()

23/12/27 16:20:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-277bae75-adc0-4984-a4e1-70135aa56e64. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/27 16:20:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
#awaiting the source termination
streamingquery.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|words|count|
+-----+-----+
+-----+-----+



23/12/27 16:21:08 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 13868 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+----------+-----+
|     words|count|
+----------+-----+
|     input|    1|
|      code|    1|
|     words|    1|
|  properly|    1|
|     count|    1|
| streaming|    1|
|appliation|    1|
|        is|    1|
|       the|    2|
|structured|    1|
|       Hii|    1|
|        of|    1|
|   running|    1|
|   working|    1|
|    number|    1|
+----------+-----+



23/12/27 16:21:16 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 8243 milliseconds
23/12/27 16:21:48 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 5060 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+----------+-----+
|     words|count|
+----------+-----+
|     input|    1|
|      code|    1|
|     words|    1|
|  properly|    1|
|     count|    1|
| streaming|    1|
|appliation|    1|
|        is|    1|
|       the|    2|
|structured|    1|
|       Hii|    1|
|        of|    1|
|   running|    1|
|   working|    1|
|    number|    2|
+----------+-----+



23/12/27 16:21:53 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 5011 milliseconds


-------------------------------------------
Batch: 3
-------------------------------------------
+----------+-----+
|     words|count|
+----------+-----+
|     input|    1|
|      code|    1|
|     words|    2|
|  properly|    1|
|     count|    1|
| streaming|    1|
|appliation|    1|
|        is|    1|
|       the|    2|
|structured|    1|
|       Hii|    2|
|        of|    1|
|   running|    1|
|   working|    1|
|    number|    2|
+----------+-----+

